In [1]:
import cv2
import mediapipe as mp
import time
import os

# Inicializar MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Carpeta donde guardar las fotos
output_dir = "capturas"
os.makedirs(output_dir, exist_ok=True)
foto_id = 0

# Capturar video de la webcam
cap = cv2.VideoCapture(1)

# Verificar si la cámara se abrió correctamente
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara")
    exit()

while True:
    # Leer frame
    success, img = cap.read()
    if not success:
        print("Error al leer el frame")
        break

    # Convertir la imagen a RGB
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Procesar la imagen
    results = hands.process(imgRGB)

    # Dibujar las marcas y el cuadro si se detectan manos
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Dibujar los puntos y conexiones
            mp_draw.draw_landmarks(
                img,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )
            # Calcular y dibujar el bounding box
            h, w, _ = img.shape
            x_list = []
            y_list = []
            for lm in hand_landmarks.landmark:
                x_list.append(int(lm.x * w))
                y_list.append(int(lm.y * h))
            x_min, x_max = min(x_list)-10, max(x_list)+10
            y_min, y_max = min(y_list)-10, max(y_list)+10
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Mostrar la imagen
    cv2.imshow("Hand Tracking", img)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == 32:  # Barra espaciadora
        filename = os.path.join(output_dir, f"foto_{foto_id}.jpg")
        cv2.imwrite(filename, img)
        print(f"Foto guardada: {filename}")
        foto_id += 1

# Liberar recursos
cap.release()
cv2.destroyAllWindows()